# Investigation of the Boston House Price dataset

Dataset has 506 instances to work with and has 14 attributes including the output attribute MEDV.

We can see that many of the attributes have a strong correlation (e.g.Attributes > 0:70 or Attributes < 0:70). For example:
- NOX and INDUS with 0.77.
- DIS and INDUS with -0.71.
- TAX and INDUS with 0.72.
- AGE and NOX with 0.73.
- DIS and NOX with -0.78.
It also looks like LSTAT has a good negative correlation with the output variable MEDV with
value of -0.74.

In [126]:
############################## Load Libraries #####################################
from numpy import arange
from numpy import array
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

############################## Import Dataset #####################################
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO','B','LSTAT', 'MEDV']
dataset = read_csv('C:/Users/Satish/python_files/Bhousing.csv', delim_whitespace=True, names=names)

############################## Import Dataset #####################################
# print(dataset.head(5))
print(dataset.shape)
set_option('precision',2)
# print(dataset.describe())
# print(dataset.corr(method='pearson'))
# print(dataset.skew())
# print(dataset.dtypes)

############################## Data Visualize #####################################


# dataset.hist(sharex=False, sharey=False, xlabelsize= 1, ylabelsize= 1)
### some attributes may have an exponential distribution, such as CRIM, ZN, Dis, Age
### some attributes may have an Binomial distribution, such as Lstat, RAD

# dataset.plot(kind='density', subplots=True, sharex=False, fontsize=1, layout=(4,4))
### looks like NOX, RM and LSTAT may be skewed Gaussian distributions.

# dataset.plot(kind='box', subplots=True, sharex=False, sharey=False, layout=(4,4), fontsize=8)

# scatter_matrix(dataset)
# pyplot.show()

#correlationmatrix
# fig = pyplot.figure()
# ax = fig.add_subplot(111)
# cax = ax.matshow(dataset.corr(), vmin=-1, vmax=1, interpolation='none')
# fig.colorbar(cax)
# ticks = numpy.arange(0,14,1)
# ax.set_xticks(ticks)
# ax.set_yticks(ticks)
# ax.set_xticklabels(names)
# ax.set_yticklabels(names)
# pyplot.show()

### The dark yellow color shows positive correlation whereas the dark blue color shows negative
### correlation. It also suggest that dark yellow and blue good candidates for removal to 
### better improve accuracy of models.

############################## Create train and test Dataset #####################################

array = dataset.values
# print(array[0:10,0:13])
x = array[:,0:13]
y = array[:,13]
# print(y[:5])

validation_size = 0.20
num_folds = 10
seed = 17
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=validation_size, random_state = seed)
scoring = 'neg_mean_squared_error'

############################## Build the model #####################################
models = []
models.append(('LR',LinearRegression()))
models.append(('LSS', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVM', SVR()))

############################## Evaluate each model #####################################
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
#     print(name, cv_results.mean(), cv_results.std())
#### LR and CART has lowest MSE

############################## Compare the model #######################################

# fig = pyplot.figure()
# fig.suptitle('AlgorithmComparison')
# ax = fig.add_subplot(111)
# pyplot.boxplot(results)
# ax.set_xticklabels(names)
# pyplot.show()

############################## Standardize the dataset #####################################
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))                 
pipelines.append(('ScaledLSS', Pipeline([('Scaler', StandardScaler()),('LSS', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledSVM', Pipeline([('Scaler', StandardScaler()),('SVM', SVR())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
#     print(name, cv_results.mean(), cv_results.std())
### we can see that scaling has effect on KNN, bringing down the error lower than other models    
   
############################## Compare the Scaled model #######################################    
# fig = pyplot.figure()
# fig.suptitle('Scaled Algorithm Comparison')
# ax = fig.add_subplot(111)
# pyplot.boxplot(results)
# ax.set_xticklabels(names)
# pyplot.show()    
###KNN has both a tight distribution of error and has the lowest score.

############################## Improve result with Tuning #######################################
### KNN Algo tunning
scaler = StandardScaler().fit(x_train)
rescaledx = scaler.transform(x_train)
k_values = numpy.array([1,3,5,7,9,11,13,15,17,19,21])
param_grid = dict(n_neighbors = k_values)
model = KNeighborsRegressor()
kfold = KFold(n_splits=num_folds,random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold, scoring=scoring)
grid_result = grid.fit(rescaledx, y_train)
# print("Best Score: ",grid_result.best_score_, "Best param: ", grid_result.best_params_)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print(mean, stdev, param)
### we can see that KNN = 3 provide MSE =  -20.659 wich is best so far now.

################## Improve the performance of model using Ensamble methods ##########################
ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()), ('AB', AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()), ('GBM', GradientBoostingRegressor())])))
ensembles.append(('ScaledRM', Pipeline([('Scaler', StandardScaler()), ('RM', RandomForestRegressor())])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()), ('ET', ExtraTreesRegressor())])))

results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
#     print(name, cv_results.mean(), cv_results.std())

# fig = pyplot.figure()
# fig.suptitle('Scaled Ensemble Algorithm Comparison')
# ax = fig.add_subplot(111)
# pyplot.boxplot(results)
# ax.set_xticklabels(names)
# pyplot.show() 
### It looks like Gradient Boosting has a better mean score.

############################## Tune Ensemble Mothods #######################################
scaler = StandardScaler().fit(x_train)
rescaledx = scaler.transform(x_train)
param_grid = dict(n_estimators = numpy.array([50,100,150,200,250,300,350,400]))
model  = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold, scoring=scoring)
grid_result = grid.fit(rescaledx, y_train)

#### Summerize the best performance and check how performance changed with esch differenct configuration

print("Best: ", grid_result.best_score_, "using: ", grid_result.best_params_)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
# for mean, std, param in zip(means, stds, params):
#     print(mean, std, param)
### We can see that the best configuration was n estimators=350 resulting in a mean squared error of -10.14
    
############################## Finalize the model #######################################
scaler = StandardScaler().fit(x_train)
rescaledx = scaler.transform(x_train)
model = GradientBoostingRegressor(random_state=seed, n_estimators=350)
model.fit(rescaledx, y_train)

############################## Transform the test dataset #######################################
rescaledtest_x = scaler.transform(x_test)
prediction = model.predict(rescaledtest_x)
print(mean_squared_error(y_test, prediction))
######### mean_squared_error : 8.158

(506, 14)
Best:  -10.140050179779665 using:  {'n_estimators': 350}
8.158339502671137
